In [1]:
import sys
import os


In [2]:
sys.path.append('..')

In [3]:
from utils import *

In [4]:
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
display(grid2values(diag_sudoku_grid))
values = grid2values(diag_sudoku_grid)

    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678

In [5]:
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units

In [6]:
'123456789'[::-1]

'987654321'

In [7]:
# TODO: Update the unit list to add the new diagonal units
d1 = [''.join(ln) for ln in list(zip('ABCDEFGHI','123456789'))]
d2 = [''.join(ln) for ln in list(zip('ABCDEFGHI','123456789'[::-1]))]
print(d1)
print(d2)

['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9']
['A9', 'B8', 'C7', 'D6', 'E5', 'F4', 'G3', 'H2', 'I1']


In [8]:
unitlist.append(d1)
unitlist.append(d2)

In [9]:
# Must be called after all units (including diagonals) are added to the unitlist
units = extract_units(unitlist, boxes)
peers = extract_peers(units, boxes)

In [10]:
units

defaultdict(list,
            {'A1': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
              ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
              ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'],
              ['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9']],
             'A2': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
              ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
              ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']],
             'A3': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
              ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
              ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']],
             'A4': [['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
              ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
              ['A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6']],
             'A5': [['A1', 'A2', 'A3', 'A4',

In [11]:
peers

defaultdict(set,
            {'A1': {'A2',
              'A3',
              'A4',
              'A5',
              'A6',
              'A7',
              'A8',
              'A9',
              'B1',
              'B2',
              'B3',
              'C1',
              'C2',
              'C3',
              'D1',
              'D4',
              'E1',
              'E5',
              'F1',
              'F6',
              'G1',
              'G7',
              'H1',
              'H8',
              'I1',
              'I9'},
             'A2': {'A1',
              'A3',
              'A4',
              'A5',
              'A6',
              'A7',
              'A8',
              'A9',
              'B1',
              'B2',
              'B3',
              'C1',
              'C2',
              'C3',
              'D2',
              'E2',
              'F2',
              'G2',
              'H2',
              'I2'},
             'A3': {'A1',
              'A2',
   

In [12]:
def get_peers(box):
    peers = set(peer for unit in unitlist if box in unit for peer in unit)
    peers.remove(box)
    return peers

In [13]:
p1 = get_peers('E5')
p2 = peers['E5']

In [14]:
print('{} -- {}'.format(type(p1), type(p2)))

<class 'set'> -- <class 'set'>


In [15]:
p1 == p2

True

Both approaches are the same

In [16]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    new_values = values.copy()
    for key, value in values.items():
        # print('key: {}, value: {}'.format(key, value))
        if len(value) == 1:
            for peer in peers[key]:
                new_values[peer] = new_values[peer].replace(value[0], '')
    return new_values

In [17]:
display(values)

    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678

In [18]:
display(eliminate(values))

   2     356789  34789  |1345789  134578  134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [19]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    def confirm_digit(values, unit_keys, digit):
        for key in unit_keys:
            if digit in values[key]:
                values[key] = digit
    return values
    
    all_digits = '123456789'

    for unit_keys in unitlist:
        unit_digits = ''.join([values[box] for box in unit_keys])
        for digit in all_digits:
            if unit_digits.count(digit) == 1:
                values = confirm_digit(values, unit_keys, digit)
    return values

In [20]:
display(only_choice(eliminate(values)))

   2     356789  34789  |1345789  134578  134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [21]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)

        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        if stalled:  # This includes the case where the puzzle is solved
            break
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [22]:
display(reduce_puzzle(values))

   2     356789  34789  |1345789  134578  134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [23]:
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if not values:
        return False
    # Choose one of the unfilled squares with the fewest possibilities
    num_choices = {(box, len(v)) for box, v in values.items() if len(v) > 1}
    if num_choices == set():
        return values
    min_box = min(num_choices, key = lambda x:x[1])[0]
    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), return that answer!
    for value in values[min_box]:
        temp_values = values.copy()
        temp_values[min_box] = value
        temp_values = search(temp_values)
        if temp_values:
            return temp_values

In [24]:
display(search(values))

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 



In [25]:
import solution as sol

In [26]:
diagonal_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'

In [27]:

display(search(grid2values(diagonal_grid)))

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 



In [28]:
solved_diag_sudoku = {'G7': '8', 'G6': '9', 'G5': '7', 'G4': '3', 'G3': '2', 'G2': '4', 'G1': '6', 'G9': '5',
                          'G8': '1', 'C9': '6', 'C8': '7', 'C3': '1', 'C2': '9', 'C1': '4', 'C7': '5', 'C6': '3',
                          'C5': '2', 'C4': '8', 'E5': '9', 'E4': '1', 'F1': '1', 'F2': '2', 'F3': '9', 'F4': '6',
                          'F5': '5', 'F6': '7', 'F7': '4', 'F8': '3', 'F9': '8', 'B4': '7', 'B5': '1', 'B6': '6',
                          'B7': '2', 'B1': '8', 'B2': '5', 'B3': '3', 'B8': '4', 'B9': '9', 'I9': '3', 'I8': '2',
                          'I1': '7', 'I3': '8', 'I2': '1', 'I5': '6', 'I4': '5', 'I7': '9', 'I6': '4', 'A1': '2',
                          'A3': '7', 'A2': '6', 'E9': '7', 'A4': '9', 'A7': '3', 'A6': '5', 'A9': '1', 'A8': '8',
                          'E7': '6', 'E6': '2', 'E1': '3', 'E3': '4', 'E2': '8', 'E8': '5', 'A5': '4', 'H8': '6',
                          'H9': '4', 'H2': '3', 'H3': '5', 'H1': '9', 'H6': '1', 'H7': '7', 'H4': '2', 'H5': '8',
                          'D8': '9', 'D9': '2', 'D6': '8', 'D7': '1', 'D4': '4', 'D5': '3', 'D2': '7', 'D3': '6',
                          'D1': '5'}

In [29]:
display(solved_diag_sudoku)

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 



In [30]:
s1 = search(grid2values(diagonal_grid))
s1 == solved_diag_sudoku

True

## Naked twins strategy

In [31]:
unitlist

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
 ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
 ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
 ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
 ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9',

In [32]:
unit = unitlist[18]
unit

['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

In [33]:
v2 = only_choice(eliminate(values))
v2['A1'] = '57'
display(v2)

   57    356789  34789  |1345789  134578  134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [34]:
twin = (('A1', 'B2'), '57')
twin[0][0]

'A1'

In [35]:
unit

['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

In [36]:
not_twins = list(set(unit) - set(twin[0]))
not_twins

['A2', 'B1', 'C3', 'C1', 'B3', 'C2', 'A3']

In [37]:
for box in not_twins:
    v2[box] = v2[box].translate(str.maketrans('', '', twin[1]))

In [38]:
display(v2)

   57     3689    3489  |1345789  134578  134579 | 13569  1345689   145   
  489      57     3489  |1345789  134578    6    |   2      1345   14589  
  4689    3689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [39]:
def eliminate_non_twins(values, unit, twin):
    temp_values = values.copy()
    not_twins = list(set(unit) - set(twin[0]))
    for box in not_twins:
        temp_values[box] = temp_values[box].translate(str.maketrans('', '', twin[1]))
    return temp_values

In [40]:
unit

['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

In [41]:
twin

(('A1', 'B2'), '57')

In [42]:
display(eliminate_non_twins(v2, unit, twin))

   57     3689    3489  |1345789  134578  134579 | 13569  1345689   145   
  489      57     3489  |1345789  134578    6    |   2      1345   14589  
  4689    3689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [43]:
v2 = only_choice(eliminate(values))
v2['A1'] = '57'
v2['A3'] = '98'
v2['A5'] = '31'
display(v2)

   57    356789    98   |1345789    31    134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [44]:
unit

['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']

In [45]:
l2_boxes = [box for box in unit if len(v2[box])==2]
l2_boxes

['A1', 'A3', 'B2']

In [46]:
import itertools

In [47]:
pairs = list(itertools.combinations(l2_boxes, 2))
pairs

[('A1', 'A3'), ('A1', 'B2'), ('A3', 'B2')]

In [48]:
twins = [((b1, b2), v2[b1]) for b1, b2 in pairs if v2[b1] == v2[b2]]
twins

[(('A1', 'B2'), '57')]

In [49]:
for twin in twins:
    v2 = eliminate_non_twins(v2, unit, twin)
display(v2)

   57     3689     98   |1345789    31    134579 | 13569  1345689   145   
  489      57     3489  |1345789  134578    6    |   2      1345   14589  
  4689    3689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [50]:
def eliminate_unit(values, unit):
        temp_values = values.copy()
        l2_boxes = [box for box in unit if len(values[box])==2]
        pairs = list(itertools.combinations(l2_boxes, 2))
        twins = [((b1, b2), values[b1]) for b1, b2 in pairs if values[b1] == values[b2]]
        for twin in twins:
            temp_values = eliminate_non_twins(temp_values, unit, twin)
        return temp_values

In [51]:
v2 = only_choice(eliminate(values))
v2['A1'] = '57'
v2['A3'] = '98'
v2['A5'] = '31'
display(v2)

   57    356789    98   |1345789    31    134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [52]:
display(eliminate_unit(v2, unit))

   57     3689     98   |1345789    31    134579 | 13569  1345689   145   
  489      57     3489  |1345789  134578    6    |   2      1345   14589  
  4689    3689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2     134678  13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [53]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.

    The naked twins strategy says that if you have two or more unallocated boxes
    in a unit and there are only two digits that can go in those two boxes, then
    those two digits can be eliminated from the possible assignments of all other
    boxes in the same unit.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the naked twins eliminated from peers

    Notes
    -----
    Your solution can either process all pairs of naked twins from the input once,
    or it can continue processing pairs of naked twins until there are no such
    pairs remaining -- the project assistant test suite will accept either
    convention. However, it will not accept code that does not process all pairs
    of naked twins from the original input. (For example, if you start processing
    pairs of twins and eliminate another pair of twins before the second pair
    is processed then your code will fail the PA test suite.)

    The first convention is preferred for consistency with the other strategies,
    and because it is simpler (since the reduce_puzzle function already calls this
    strategy repeatedly).
    """
    def eliminate_non_twins(values, unit, twin):
        temp_values = values.copy()
        not_twins = list(set(unit) - set(twin[0]))
        for box in not_twins:
            values[box] = values[box].translate(str.maketrans('', '', twin[1]))
        return values
    
    def eliminate_unit(values, unit):
        l2_boxes = [box for box in unit if len(values[box])==2]
        pairs = list(itertools.combinations(l2_boxes, 2))
        twins = [((b1, b2), values[b1]) for b1, b2 in pairs if values[b1] == values[b2]]
        for twin in twins:
            values = eliminate_non_twins(values, unit, twin)
        return values
    
    for unit in unitlist:
        print(unit)
        values = eliminate_unit(values, unit)
        display(values)
    return values

In [54]:
v2 = only_choice(eliminate(values))
v2['A1'] = '57'
v2['A3'] = '98'
v2['A5'] = '31'
v2['H5'] = '31'
display(v2)

   57    356789    98   |1345789    31    134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2       31    13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    



In [55]:
display(naked_twins(v2))

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
   57    356789    98   |1345789    31    134579 | 13569  1345689   145   
 45789     57    34789  |1345789  134578    6    |   2      1345   14589  
 45689   35689     1    | 34589   23458   23459  |   35      7     45689  
------------------------+------------------------+------------------------
 14579   12579     6    |  457    123457    8    |  1359   12359    1259  
   3      1258    248   |  145      9      1245  |  156    12568     7    
 15789   125789   2789  |   6     12357     57   |   4     123589  12589  
------------------------+------------------------+------------------------
  1679     4      237   | 13579   13567   13579  |   8     12569   12569  
 16789    137      5    |   2       31    13479  |  1679     46     1469  
   17    126789   2789  | 145789  145678  14579  | 15679   124569    3    

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9']
   57    356789    98   |1345789    31    134579 | 13569  134568